# Hydraulic Model

In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines

import seaborn as sns
import pandas as pd

import json

from striprtf.striprtf import rtf_to_text
import re

# import function from another jupyter notebook
import import_ipynb
import Function_simple
import Function_error

### Inputs
For varying soil matric potentials hb and transpiration rates E we calculate the leaf water potential hleaf

In [2]:
dx, dy, dz = 100, 100, 0.3

h_b = -1*np.arange(10,15010+100,dx, dtype=int) #bulk soil water potential in cm heads
E = 3e-6*np.arange(0,3000+1,dz) #transpiration rate in cm3 s**-1

Import SOL data to fit

In [3]:
with open('../2_Data/1_SoilData/soil_Data.json') as f:
    soil_Data = json.load(f)

In [4]:
soil_types_interested = ["Clay", "Silt Loam", "Sandy Loam"]
soil_Data = {soil: soil_Data[soil] for soil in soil_types_interested}

### Optimization

Import parameters found on Cluster Code

In [5]:
def parse_soil_data(text):
    data = {}
    current_soil = None

    # Regular expression patterns
    section_header_pattern = re.compile(r'^([A-Za-z ]+):$')
    key_value_pattern = re.compile(r'^(\w+):\s*\((.*?)\)$')
    float_pattern = re.compile(r'np\.float64\(([-\d.eE+]+)\)')

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        # Detect new soil type
        header_match = section_header_pattern.match(line)
        if header_match:
            current_soil = header_match.group(1)
            data[current_soil] = {}
            continue

        # Detect key-value pair
        key_val_match = key_value_pattern.match(line)
        if key_val_match and current_soil:
            key = key_val_match.group(1)
            float_values = float_pattern.findall(key_val_match.group(2))
            data[current_soil][key] = [float(val) for val in float_values]

    return data

In [6]:
with open("../3_Results/Optimization/Optimization_Euler_Results_newmethod.rtf") as f:
    content = f.read()
    text = rtf_to_text(content)

In [7]:
data_opt_params = parse_soil_data(text)

In [8]:
for soil in data_opt_params:
    opt_R_root = np.mean(data_opt_params[soil]['R_root'])
    opt_h0_x = np.mean(data_opt_params[soil]['h0_x'])
    opt_tau_x = np.mean(data_opt_params[soil]['tau_x'])

    opt_params = [opt_R_root, opt_h0_x, opt_tau_x]

    data_opt_params[soil]['opt_params'] = opt_params

In [ ]:
""" data_opt_params_json = {soil: {param: values.tolist() for param, values in params.items()} 
                        for soil, params in data_opt_params.items()} 
                        
with open("data_opt_params.json", "w") as f:
    json.dump(data_opt_params_json, f, indent=4) """

In [ ]:
with open('../2_Data/Opt/data_opt_params_newmethod.json') as f:
    data_opt_params = json.load(f)

In [ ]:
data_opt_params_json = {soil: {param: values for param, values in params.items()} 
                        for soil, params in data_opt_params.items()} 
                        
with open("../2_Data/Opt/data_opt_params_newmethod.json", "w") as f:
    json.dump(data_opt_params_json, f, indent=4)

#### Run the model with the optimal parameters
For each set of optimal parameters, run the model and calculate the SOL

In [9]:
transpiration = {}

for soil in data_opt_params:
    print(soil)

    opt_params = data_opt_params[soil]['opt_params']

    SOL_opt = Function_simple.Calc_SOL(E, h_b, opt_params)

    trajecthleaf1 = Function_simple.Calc_hleaf(E, -10, opt_params) # Calculate the well-watered curve

    print(opt_params)

    soil_SOL = np.array(soil_Data[soil]["traject"])
    
    variation = np.sum(np.abs(np.diff(SOL_opt[:,2])))

    error = Function_error.calc_error(SOL_opt, soil_SOL) # calculate error

    transpiration[soil] = {
        "trajectleaf1": trajecthleaf1,
        "SOL": SOL_opt,
        "RMSE": error
        }

Clay
[np.float64(1278963.2313953512), np.float64(-21620.229407422165), np.float64(8.007718501029313)]
Silt Loam
[np.float64(1279819.93830479), np.float64(-21030.552903509764), np.float64(7.727403093037177)]
Sandy Loam
[np.float64(1190010.8034856948), np.float64(-15626.643607193439), np.float64(7.697114664262647)]


### Outputs
Put everything on the same plot

In [ ]:
# find the mask to limit curves on the plot
def mask(traject, minlim, maxlim):
    return (-traject >= minlim) & (-traject <= maxlim)

Plot only complex SOL

In [ ]:
sns.set_style('white')

labels = ['a', 'b', 'c']

cmap = sns.cubehelix_palette(start=0., rot=-.75, light=0.6, dark=0.2, n_colors=3)
cmap_contrast = sns.cubehelix_palette(start=0.5, rot=1.5, light=.75, dark=0.1, n_colors=4) # colormap contrast

color = cmap[1]
color_fit = cmap[0]
color_contrast = cmap_contrast[1]

line_bold = 2.5
line_thin = 1.
fontsize=14

plt.figure(1)
fig, axs = plt.subplots(2, 3, figsize=(10, 6), sharex='row', sharey='row')
plt.subplots_adjust(hspace=0.4)

for i, soil in enumerate(transpiration):
    soil_SOL = np.array(soil_Data[soil]["traject"])

    # leaf trajectories
    trajecthleaf1_opt = np.array(transpiration[soil]["trajectleaf1"])
    trajecthleaf1 = np.array(soil_Data[soil]["trajecthleaf_-10"])
    trajecthleaf30 = np.array(soil_Data[soil]["trajecthleaf_-3000"])
    trajecthleaf50 = np.array(soil_Data[soil]["trajecthleaf_-5000"])
    trajecthleaf70 = np.array(soil_Data[soil]["trajecthleaf_-7000"])
    trajecthleaf100 = np.array(soil_Data[soil]["trajecthleaf_-10000"])
    trajecthleaf150 = np.array(soil_Data[soil]["trajecthleaf_-15000"])

    # soil trajectories
    trajecthsoil2000 = np.array(soil_Data[soil]["trajecthsoil_-2000"])
    trajecthsoil5000 = np.array(soil_Data[soil]["trajecthsoil_-5000"])
    trajecthsoil7500 = np.array(soil_Data[soil]["trajecthsoil_-7500"])
    trajecthsoil10000 = np.array(soil_Data[soil]["trajecthsoil_-10000"])
    trajecthsoil15000 = np.array(soil_Data[soil]["trajecthsoil_-15000"])
    trajecthsoil30000 = np.array(soil_Data[soil]["trajecthsoil_-30000"])


    # LEAF PLOT
    ax = axs[0, i]

    minleaf = 0*1e4
    maxleaf = 2.3*1e4
    trajecthleaf1_mask = mask(trajecthleaf1, minleaf, maxleaf)
    trajecthleaf1_opt_mask = mask(trajecthleaf1_opt, minleaf, maxleaf)
    trajecthleaf30_mask = mask(trajecthleaf30, minleaf, maxleaf)
    trajecthleaf50_mask = mask(trajecthleaf50, minleaf, maxleaf)
    trajecthleaf70_mask = mask(trajecthleaf70, minleaf, maxleaf)
    trajecthleaf100_mask = mask(trajecthleaf100, minleaf, maxleaf)
    trajecthleaf150_mask = mask(trajecthleaf150, minleaf, maxleaf)

    ax.plot(-trajecthleaf1[trajecthleaf1_mask] * 1e-4, E[trajecthleaf1_mask] / np.nanmax(E), 'k', linewidth=line_bold, label='well-watered')
    ax.plot(-trajecthleaf30[trajecthleaf30_mask] * 1e-4, E[trajecthleaf30_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf50[trajecthleaf50_mask] * 1e-4, E[trajecthleaf50_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf70[trajecthleaf70_mask] * 1e-4, E[trajecthleaf70_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf100[trajecthleaf100_mask] * 1e-4, E[trajecthleaf100_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf150[trajecthleaf150_mask] * 1e-4, E[trajecthleaf150_mask] / np.nanmax(E), 'k--', linewidth=line_thin, label='iso-lines '+r'$\psi_{s,b}$')

    # sol
    ax.plot(-soil_SOL[:, 1] * 1e-4, soil_SOL[:, 2] / np.nanmax(E), color=color, linewidth=line_bold, label='SOL complex')
    ax.set_xlabel(r'$-\psi_{leaf} \ (MPa)$', fontsize=fontsize)
    ax.tick_params(bottom=True)
    #ax.set_xlim(0.5, 2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    #ax.set_xbound(lower=-0.1, upper=3)

    background_color = ax.get_facecolor()

    # SOIL PLOT
    ax2 = axs[1, i]

    ax2.plot(-trajecthsoil7500[:, 0] * 1e-4, trajecthsoil7500[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil2000[:, 0] * 1e-4, trajecthsoil2000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil5000[:, 0] * 1e-4, trajecthsoil5000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil10000[:, 0] * 1e-4, trajecthsoil10000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil15000[:, 0] * 1e-4, trajecthsoil15000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil30000[:, 0] * 1e-4, trajecthsoil30000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed', label='iso-lines '+r'$\psi_{leaf}$')
    
    # sol
    ax2.plot(-soil_SOL[:, 0] * 1e-4, soil_SOL[:, 2] / np.nanmax(E), color=color, linewidth=line_bold, label="SOL complex")
    ax2.set_xlabel(r'$-\psi_{s,b} \ (MPa)$', fontsize=fontsize)
    ax2.tick_params(bottom=True)

    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)

    # add axis labels, legend, and labelling
    if i == 0:
        ax.text(-0.1, 1.15, labels[0], transform=ax.transAxes, va='top', ha='right', fontsize=fontsize+2, weight='bold')
        ax2.text(-0.1, 1.12, labels[1], transform=ax2.transAxes, va='top', ha='right', fontsize=fontsize+2, weight='bold')

    # display title and error
    axs[0, i].set_title(soil, weight='bold', fontsize=fontsize, pad=10) # title with soil name

    axs[0, 0].set_ylabel(r'$E \ [-]$', fontsize=fontsize)
    axs[0, 0].tick_params(left=True)
    axs[1, 0].set_ylabel(r'$E \ [-]$', fontsize=fontsize)
    axs[1, 0].tick_params(left=True)

    handles, labels = axs[0, 1].get_legend_handles_labels()
    axs[0, 2].legend(handles, labels, loc='center left', bbox_to_anchor=(1., 0.5, 0.8, .01), fontsize=fontsize, frameon=False) 

    handles, labels = axs[1, 1].get_legend_handles_labels()
    axs[1, 2].legend(handles, labels, loc='center left', bbox_to_anchor=(1., 0.5, 0.8, .01), fontsize=fontsize, frameon=False) 

plt.tight_layout()
plt.savefig('comparison_SOL.png', dpi=300, bbox_inches='tight')
plt.show()

Plot with simple SOL

In [ ]:
sns.set_style('white')

labels = ['a', 'b', 'c']

cmap = sns.cubehelix_palette(start=0., rot=-.75, light=0.6, dark=0.2, n_colors=3)
cmap_contrast = sns.cubehelix_palette(start=0.5, rot=1.5, light=.75, dark=0.1, n_colors=4) # colormap contrast

color = cmap[1]
color_fit = cmap[0]
color_contrast = cmap_contrast[1]

line_bold = 2.5
line_thin = 1.
fontsize=14

plt.figure(1)
fig, axs = plt.subplots(3, 3, figsize=(10, 10), sharex='row', sharey='row')
plt.subplots_adjust(hspace=0.4)

for i, soil in enumerate(transpiration):
    trajectory2 = transpiration[soil]["SOL"]
    soil_SOL = np.array(soil_Data[soil]["traject"])
    error = transpiration[soil]["RMSE"]

    # leaf trajectories
    trajecthleaf1_opt = np.array(transpiration[soil]["trajectleaf1"])
    trajecthleaf1 = np.array(soil_Data[soil]["trajecthleaf_-10"])
    trajecthleaf30 = np.array(soil_Data[soil]["trajecthleaf_-3000"])
    trajecthleaf50 = np.array(soil_Data[soil]["trajecthleaf_-5000"])
    trajecthleaf70 = np.array(soil_Data[soil]["trajecthleaf_-7000"])
    trajecthleaf100 = np.array(soil_Data[soil]["trajecthleaf_-10000"])
    trajecthleaf150 = np.array(soil_Data[soil]["trajecthleaf_-15000"])

    # soil trajectories
    trajecthsoil2000 = np.array(soil_Data[soil]["trajecthsoil_-2000"])
    trajecthsoil5000 = np.array(soil_Data[soil]["trajecthsoil_-5000"])
    trajecthsoil7500 = np.array(soil_Data[soil]["trajecthsoil_-7500"])
    trajecthsoil10000 = np.array(soil_Data[soil]["trajecthsoil_-10000"])
    trajecthsoil15000 = np.array(soil_Data[soil]["trajecthsoil_-15000"])
    trajecthsoil30000 = np.array(soil_Data[soil]["trajecthsoil_-30000"])

    # transpiration trajectories
    trajectE01 = np.array(soil_Data[soil]["trajectE_0.1"])
    trajectE02 = np.array(soil_Data[soil]["trajectE_0.2"])
    trajectE04 = np.array(soil_Data[soil]["trajectE_0.4"])
    trajectE06 = np.array(soil_Data[soil]["trajectE_0.6"])
    trajectE08 = np.array(soil_Data[soil]["trajectE_0.8"])


    # LEAF PLOT
    ax = axs[0, i]

    minleaf = 0*1e4
    maxleaf = 2.3*1e4
    trajecthleaf1_mask = mask(trajecthleaf1, minleaf, maxleaf)
    trajecthleaf1_opt_mask = mask(trajecthleaf1_opt, minleaf, maxleaf)
    trajecthleaf30_mask = mask(trajecthleaf30, minleaf, maxleaf)
    trajecthleaf50_mask = mask(trajecthleaf50, minleaf, maxleaf)
    trajecthleaf70_mask = mask(trajecthleaf70, minleaf, maxleaf)
    trajecthleaf100_mask = mask(trajecthleaf100, minleaf, maxleaf)
    trajecthleaf150_mask = mask(trajecthleaf150, minleaf, maxleaf)

    ax.plot(-trajecthleaf1[trajecthleaf1_mask] * 1e-4, E[trajecthleaf1_mask] / np.nanmax(E), 'k', linewidth=line_bold, label='well-watered')
    ax.plot(-trajecthleaf1_opt[trajecthleaf1_opt_mask] * 1e-4, E[trajecthleaf1_opt_mask] / np.nanmax(E), 'k--', linewidth=line_bold, label='well-w. simple')
    ax.plot(-trajecthleaf30[trajecthleaf30_mask] * 1e-4, E[trajecthleaf30_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf50[trajecthleaf50_mask] * 1e-4, E[trajecthleaf50_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf70[trajecthleaf70_mask] * 1e-4, E[trajecthleaf70_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf100[trajecthleaf100_mask] * 1e-4, E[trajecthleaf100_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
    ax.plot(-trajecthleaf150[trajecthleaf150_mask] * 1e-4, E[trajecthleaf150_mask] / np.nanmax(E), 'k--', linewidth=line_thin, label='iso-lines '+r'$\psi_{s,b}$')

    # sol
    ax.plot(-soil_SOL[:, 1] * 1e-4, soil_SOL[:, 2] / np.nanmax(E), color=color, linewidth=line_bold, label='SOL complex')
    ax.plot(-trajectory2[:, 1] * 1e-4, trajectory2[:, 2] / np.nanmax(E), color=color_fit, linestyle='dashed', linewidth=line_bold, label='SOL simple')
    ax.set_xlabel(r'$-\psi_{leaf} \ (MPa)$', fontsize=fontsize)
    ax.tick_params(bottom=True)
    #ax.set_xlim(0.5, 2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    #ax.set_xbound(lower=-0.1, upper=3)

    background_color = ax.get_facecolor()

    # SOIL PLOT
    ax2 = axs[1, i]

    ax2.plot(-trajecthsoil7500[:, 0] * 1e-4, trajecthsoil7500[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil2000[:, 0] * 1e-4, trajecthsoil2000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil5000[:, 0] * 1e-4, trajecthsoil5000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil10000[:, 0] * 1e-4, trajecthsoil10000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil15000[:, 0] * 1e-4, trajecthsoil15000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed')
    ax2.plot(-trajecthsoil30000[:, 0] * 1e-4, trajecthsoil30000[:, 1] / np.nanmax(E), 'k', linewidth=line_thin, linestyle='dashed', label='iso-lines '+r'$\psi_{leaf}$')
    
    # sol
    ax2.plot(-soil_SOL[:, 0] * 1e-4, soil_SOL[:, 2] / np.nanmax(E), color=color, linewidth=line_bold, label="SOL complex")
    ax2.plot(-trajectory2[:, 0] * 1e-4, trajectory2[:, 2] / np.nanmax(E), color=color_fit, linestyle='dashed', linewidth=line_bold, label="SOL simple")
    ax2.set_xlabel(r'$-\psi_{s,b} \ (MPa)$', fontsize=fontsize)
    ax2.tick_params(bottom=True)

    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)

    # LEAF-SOIL PLOT
    ax3 = axs[2, i]
    
    # Define the limits for the iso lines
    min_SOL = 1.0*1e4
    max_SOL = 2.0*1e4

    ax3.plot(-h_b[mask(trajectE01, min_SOL, max_SOL)] * 1e-4, -trajectE01[mask(trajectE01, min_SOL, max_SOL)] * 1e-4, 'k', linewidth=line_thin, linestyle='dashed')
    ax3.plot(-h_b[mask(trajectE02, min_SOL, max_SOL)] * 1e-4, -trajectE02[mask(trajectE02, min_SOL, max_SOL)] * 1e-4, 'k', linewidth=line_thin, linestyle='dashed')
    ax3.plot(-h_b[mask(trajectE04, min_SOL, max_SOL)] * 1e-4, -trajectE04[mask(trajectE04, min_SOL, max_SOL)] * 1e-4, 'k', linewidth=line_thin, linestyle='dashed')
    ax3.plot(-h_b[mask(trajectE06, min_SOL, max_SOL)] * 1e-4, -trajectE06[mask(trajectE06, min_SOL, max_SOL)] * 1e-4, 'k', linewidth=line_thin, linestyle='dashed')
    ax3.plot(-h_b[mask(trajectE08, min_SOL, max_SOL)] * 1e-4, -trajectE08[mask(trajectE08, min_SOL, max_SOL)] * 1e-4, 'k', linewidth=line_thin, linestyle='dashed', label='iso-lines '+r'$E$')

    uni_leaf = np.linspace(min_SOL * 1e-4, np.max(-soil_SOL[:,0]) * 1e-4, len(soil_SOL[:,1]))

    ax3.plot(uni_leaf, uni_leaf, color=color_contrast, label=r'$\psi_{s,b}=\psi_{leaf}$', linewidth=line_bold)
    ax3.plot(-soil_SOL[:,0] * 1e-4, -soil_SOL[:,1] * 1e-4, color=color, linewidth=line_bold, label='SOL complex')
    ax3.plot(-trajectory2[:,0] * 1e-4, -trajectory2[:,1] * 1e-4, color=color_fit, linestyle='dashed', linewidth=line_bold, label='SOL simple')
    ax3.set_xlabel(r'$-\psi_{s,b} \ (MPa)$', fontsize=fontsize)
    ax3.tick_params(bottom=True)

    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)

    #ax3.set_ylim(1.0, 2.0)

    # add axis labels, legend, and labelling
    if i == 0:
        ax.text(-0.1, 1.23, labels[0], transform=ax.transAxes, va='top', ha='right', fontsize=fontsize+2, weight='bold')
        ax2.text(-0.1, 1.12, labels[1], transform=ax2.transAxes, va='top', ha='right', fontsize=fontsize+2, weight='bold')
        ax3.text(-0.1, 1.12, labels[2], transform=ax3.transAxes, va='top', ha='right', fontsize=fontsize+2, weight='bold')

    # display title and error
    axs[0, i].set_title(soil, weight='bold', fontsize=fontsize, pad=25) # title with soil name
    axs[0, i].text(0.5, 1.12, 'RMSE = {:0.3f}'.format(error), fontsize=fontsize-2, va='top', ha='center', transform=axs[0, i].transAxes) # add error for every SOL fit

    axs[0, 0].set_ylabel(r'$E \ [-]$', fontsize=fontsize)
    axs[0, 0].tick_params(left=True)
    axs[1, 0].set_ylabel(r'$E \ [-]$', fontsize=fontsize)
    axs[1, 0].tick_params(left=True)
    axs[2, 0].set_ylabel(r'$-\psi_{leaf} \ (MPa)$', fontsize=fontsize)
    axs[2, 0].tick_params(left=True)

    handles, labels = axs[0, 1].get_legend_handles_labels()
    axs[0, 2].legend(handles, labels, loc='center left', bbox_to_anchor=(1., 0.5, 0.8, .01), fontsize=fontsize, frameon=False) 

    handles, labels = axs[1, 1].get_legend_handles_labels()
    axs[1, 2].legend(handles, labels, loc='center left', bbox_to_anchor=(1., 0.5, 0.8, .01), fontsize=fontsize, frameon=False) 

    handles, labels = axs[2, 2].get_legend_handles_labels()
    axs[2, 2].legend(handles, labels, loc='center left', bbox_to_anchor=(1., 0.5, 0.8, .01), fontsize=fontsize, frameon=False) 

plt.tight_layout()
plt.savefig('comparison_all.png', dpi=300, bbox_inches='tight')
plt.show()

Compare optimal parameters

In [ ]:
# Add the values for the complex model

soil_compl = {
    'R_root': 0.12e7,
    'h0_x': -25000,
    'tau_x': 5
}

In [ ]:
# prepare dataframe

params_list = []
rows = []

# add the values from the complex soil model
for param, value in soil_compl.items():
    rows.append({'Soil': 'Complex Model', 'Parameter': param, 'Value': value})

for soil, params in data_opt_params.items():
    for param, value in params.items():
        if param != 'opt_params':
            for val in value:
                rows.append({'Soil': soil, 'Parameter': param, 'Value': val})
                params_list.append(param)

params_df = pd.DataFrame(rows)

params_df

In [ ]:
sns.set_style('white')
#sns.set_palette('ocean')

fontsize=14
#sns.set(fontsize=fontsize)

# prepare colormap
cmap = sns.cubehelix_palette(start=0., rot=-.75, light=0.8, dark=0.2, n_colors=4)
cmap = cmap[1:]

cmap_contrast = sns.cubehelix_palette(start=0.5, rot=1.5, light=.75, dark=0.1, n_colors=4) # colormap
cmap.insert(0, cmap_contrast[1])

scaling_factors = {
    'R_root': 1,
    'h0_x': -1,
    'tau_x': 1
}

params_df['Scaled Value'] = params_df.apply(
    lambda row: row['Value'] * scaling_factors.get(row['Parameter'], 1),
    axis=1
)

labels = ['a', 'b', 'c']

f, axs = plt.subplots(1, 3, figsize=(10, 3), sharex=True, sharey=False)

for i, param in enumerate(np.unique(params_list)):
    ax=axs[i]

    data = params_df[params_df['Parameter'] == param]

    #ax.bar(data=data, x='Soil', height='Scaled Value', color=color, alpha=.5)

    g = sns.barplot(data=data, x='Soil', y='Scaled Value', hue='Soil', ax=ax, legend=True, palette=cmap, estimator='mean', errorbar='sd', capsize=0.07)

    ax.xaxis.label.set_visible(False)
    ax.yaxis.label.set_visible(False)
    ax.set(xticklabels=[])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    ymin, ymax = ax.get_ylim()
    extra_space = 0.02 * (ymax - ymin)
    ax.set_ylim(ymin - extra_space, ymax)

    ax.text(-0.03, 1.07, labels[i], transform=ax.transAxes, fontsize=fontsize+2, va='center', ha='right', weight='bold') # add a and b

    if i == 0:
        ax.set_ylabel(r'$R_{root}$', fontsize=fontsize)
        ax.set_title(r'$\mathbf{R_{root}}$', fontsize=fontsize, pad=7)
        ax.ticklabel_format(axis='y', style='scientific')
    if i == 1:
        ax.set_ylabel(r'$-h_{0,x}$', labelpad=-280, fontsize=fontsize)
        ax.set_title(r'$\mathbf{-h_{0,x}}$', fontsize=fontsize, pad=7)
        ax.ticklabel_format(axis='y', style='scientific')
    if i == 2:
        ax.set_ylabel(r'$\tau_{x}$', fontsize=fontsize)
        ax.set_title(r'$\mathbf{\tau_{x}}$', fontsize=fontsize, pad=7)
    
    if i == 2:  # Only add legend to the last subplot
        ax.legend(loc='center left', fontsize=fontsize-2, title_fontsize=fontsize, bbox_to_anchor=(1.1, .5), frameon=False, alignment='left')
    else:
        ax.get_legend().remove()

#g = sns.catplot(data=params_df, kind='bar', x='Soil', y='Scaled Value', col='Parameter', sharey=False, hue='Soil')

plt.tight_layout()
plt.savefig('Comparison_parameters.png', dpi=300)
plt.show()

In [ ]:
data_opt_params['Sandy Loam']['opt_params']

Plot for Sandy Loam

In [ ]:
R_root_SL = 0.12e7
tau_x_SL = [5., 7.7, 8.3]
h0_x_SL = -1e3*np.array([12, 15.6, 21])

In [ ]:
transpiration_SL = {}

for i in range(3):
    SOL_opt = Function_simple.Calc_SOL(E, h_b, [R_root_SL, h0_x_SL[i], tau_x_SL[i]])

    soil_SOL = np.array(soil_Data['Sandy Loam']["traject"])

    error = Function_error.calc_error(SOL_opt, soil_SOL) # calculate error

    transpiration_SL[f'Version {i}'] = {
        "SOL": SOL_opt,
        "RMSE": error
        }

In [ ]:
sns.set_style('white')

#cmap = sns.cubehelix_palette(start=0., rot=-.75, light=0.6, dark=0.2, n_colors=5)
#num_elem = len(transpiration)+2
#cmap = sns.cubehelix_palette(start=.9, rot=.9, light=.55, dark=0.2, n_colors=3) # colormap
cmap = sns.cubehelix_palette(start=0., rot=-.75, light=0.8, dark=0.3, n_colors=4)
cmap = cmap[1:]
#cmap_contrast = sns.cubehelix_palette(start=0.5, rot=1.5, light=.75, dark=0.1, n_colors=5) # colormap
cmap_contrast = sns.cubehelix_palette(start=0.5, rot=1.5, light=.75, dark=0.1, n_colors=4) # colormap
cmap.insert(0, cmap_contrast[1])

color = cmap[0]

line_bold = 2.5
line_thin = 1.
fontsize = 14

plt.figure(figsize=(5, 5))

background_color = plt.gca().get_facecolor()

soil_SL = 'Sandy Loam'
soil_SOL = np.array(soil_Data[soil_SL]["traject"])

# leaf trajectories
#trajecthleaf1_opt = np.array(transpiration[soil]["trajectleaf1"])
trajecthleaf1 = np.array(soil_Data[soil_SL]["trajecthleaf_-10"])
trajecthleaf30 = np.array(soil_Data[soil_SL]["trajecthleaf_-3000"])
trajecthleaf50 = np.array(soil_Data[soil_SL]["trajecthleaf_-5000"])
trajecthleaf70 = np.array(soil_Data[soil_SL]["trajecthleaf_-7000"])
trajecthleaf100 = np.array(soil_Data[soil_SL]["trajecthleaf_-10000"])
trajecthleaf150 = np.array(soil_Data[soil_SL]["trajecthleaf_-15000"])

minleaf = 0*1e4
maxleaf = 2.5*1e4
trajecthleaf1_mask = mask(trajecthleaf1, minleaf, maxleaf)
trajecthleaf30_mask = mask(trajecthleaf30, minleaf, maxleaf)
trajecthleaf50_mask = mask(trajecthleaf50, minleaf, maxleaf)
trajecthleaf70_mask = mask(trajecthleaf70, minleaf, maxleaf)
trajecthleaf100_mask = mask(trajecthleaf100, minleaf, maxleaf)
trajecthleaf150_mask = mask(trajecthleaf150, minleaf, maxleaf)

# plot for soil data
plt.plot(-trajecthleaf1[trajecthleaf1_mask] * 1e-4, E[trajecthleaf1_mask] / np.nanmax(E), 'k', linewidth=line_bold, label='well-watered')
plt.plot(-trajecthleaf30[trajecthleaf30_mask] * 1e-4, E[trajecthleaf30_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
plt.plot(-trajecthleaf50[trajecthleaf50_mask] * 1e-4, E[trajecthleaf50_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
plt.plot(-trajecthleaf70[trajecthleaf70_mask] * 1e-4, E[trajecthleaf70_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
plt.plot(-trajecthleaf100[trajecthleaf100_mask] * 1e-4, E[trajecthleaf100_mask] / np.nanmax(E), 'k--', linewidth=line_thin)
plt.plot(-trajecthleaf150[trajecthleaf150_mask] * 1e-4, E[trajecthleaf150_mask] / np.nanmax(E), 'k--', linewidth=line_thin, label='iso-lines '+r'$\psi_{s,b}$')

plt.plot(-soil_SOL[:, 1] * 1e-4, soil_SOL[:, 2] / np.nanmax(E), color=color, linewidth=line_bold, label='SOL complex') # SOL
plt.xlabel(r'$-\psi_{leaf} \ (MPa)$', fontsize=fontsize)
plt.ylabel(r'$E \ [-]$', fontsize=fontsize)

plt.tick_params(bottom=True, left=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

alpha = 0.9 # transparency for simple model plots

for i, param in enumerate(transpiration_SL):
    trajectory2 = transpiration_SL[param]["SOL"]
    error = transpiration_SL[param]["RMSE"]

    color_fit = cmap[i+1]    

    #trajecthleaf1_opt = np.array(transpiration_SL[param]["trajectleaf1"])

    # SOL for simple model
    line, = plt.plot(-trajectory2[:, 1] * 1e-4, trajectory2[:, 2] / np.nanmax(E), color=color_fit, linewidth=line_bold, label='SOL simple', alpha=alpha, linestyle='dashed')
    line_proxy = mlines.Line2D([], [], color=color, label='SOL simple')

    # Get the x and y data
    x_data = line.get_xdata()
    y_data = line.get_ydata()

    if i == 0:
        x_pos = 0.5
        y_pos = 0.1

    if i == 1:
        x_pos = 1.6
        y_pos = 0.1

    if i == 2:
        x_pos = 1.9
        y_pos = y_data[-1]

    pos = [x_pos, y_pos]

    #ax.legend(loc='lower right', frameon=True, facecolor=background_color, framealpha=0.9, edgecolor=background_color, fontsize=fontsize)

    t = plt.text(pos[0], pos[1], '{:0.3f}'.format(error), fontsize=fontsize, color = color_fit, weight='bold', alpha=alpha, va='center', ha='left')
    t.set_bbox(dict(facecolor=background_color, edgecolor=background_color))

plt.legend(loc='center left', bbox_to_anchor=(1., 0.5, 0.8, .01), fontsize=fontsize, frameon=False)
#plt.tight_layout()
plt.savefig('comparison_SL.png', dpi=300, bbox_inches='tight')
plt.show()